## 0. House Keeping Functions

In [107]:
import csv
from scipy import ndimage

samples = []

# Assuming that DataSet are already downloaded
def prepare_samples(datasetpath):
    i = 1
    with open(datasetpath + '/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        # Skip first row
        next(reader, None)
        for line in reader:
            # Update the image paths
            image_path = datasetpath + '/IMG/' + line[0].split('/')[-1]
            line[0] = image_path
            image_path = datasetpath + '/IMG/' + line[1].split('/')[-1]
            line[1] = image_path
            image_path = datasetpath + '/IMG/' + line[2].split('/')[-1]
            line[2] = image_path
            if i == 1:
                print(line)
                i = 2
            samples.append(line)


## 1. Prepare Samples 

In [108]:
import numpy as np

# Reset the list
samples.clear()

# Extract training data
prepare_samples('Data/DataSet1')
prepare_samples('Data/DataSet2')
prepare_samples('Data/DataSet3')
prepare_samples('Data/DataSet4')

['Data/DataSet1/IMG/center_2016_12_01_13_30_48_287.jpg', 'Data/DataSet1/IMG/left_2016_12_01_13_30_48_287.jpg', 'Data/DataSet1/IMG/right_2016_12_01_13_30_48_287.jpg', ' 0', ' 0', ' 0', ' 22.14829']
['Data/DataSet2/IMG/center_2019_03_22_10_05_59_377.jpg', 'Data/DataSet2/IMG/left_2019_03_22_10_05_59_377.jpg', 'Data/DataSet2/IMG/right_2019_03_22_10_05_59_377.jpg', '0', '0', '0', '2.84E-06']
['Data/DataSet3/IMG/center_2019_03_22_09_39_12_628.jpg', 'Data/DataSet3/IMG/left_2019_03_22_09_39_12_628.jpg', 'Data/DataSet3/IMG/right_2019_03_22_09_39_12_628.jpg', '0', '0', '0', '4.49E-06']
['Data/DataSet4/IMG/center_2019_03_22_09_57_08_933.jpg', 'Data/DataSet4/IMG/left_2019_03_22_09_57_08_933.jpg', 'Data/DataSet4/IMG/right_2019_03_22_09_57_08_933.jpg', '0', '0', '0', '0.000205434']


## 2. Split Samples into '_Training Set_' & '_Validation Set_'

In [109]:
from sklearn.model_selection import train_test_split

# 80% training set and 20% validation set
Sample_train, Sample_valid = train_test_split(samples, test_size = 0.2)


## 3. Define Generator

In [110]:
import matplotlib.pyplot as plt
import sklearn    

def generator(samples, batch_size=32):
    num_samples = len(samples)
    
    while 1:  # Loop forever so the generator never terminates
        shuffle(samples)
        
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            angles = []
            
            for batch_sample in batch_samples:
                
                steer_angle = float(batch_sample[3])
                ########## Center Image ##########
                image = plt.imread(batch_sample[0])
                images.append(image)
                angles.append(steer_angle)

                # Flip the image
                images.append(np.flipud(image))
                angles.append(float( steer_angle * (-1.0) ))
                
                
                ########## left Image ##########
                image = plt.imread(batch_sample[1])
                images.append(image)
                angles.append(steer_angle + 0.2)

                # Flip the image
                images.append(np.flipud(image))
                angles.append(float( (steer_angle + 0.2) * (-1) ))
                
                
                ########## Right Image ##########
                image = plt.imread(batch_sample[2])
                images.append(image)
                angles.append(steer_angle - 0.2)
                            
                # Flip the image
                images.append(np.flipud(image))
                angles.append(float( (steer_angle - 0.2) * (-1) ))
            
            X_train = np.array(images)
            y_train = np.array(angles)
            
            yield sklearn.utils.shuffle(X_train, y_train)
            
            
            
# compile and train the model using the generator function
train_generator = generator(Sample_train, batch_size=32)
validation_generator = generator(Sample_valid, batch_size=32)

## 4. Define A Model Using Keras

In [111]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers import Lambda, Cropping2D


model = Sequential()

# Normalize the data
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))

# trim image to only see section with road
model.add(Cropping2D( cropping=((70,25),(0,0)) ) )

# Convolutional Layer 1
model.add(Conv2D(filters=24, kernel_size=5, strides=(2, 2), activation='relu'))

# Convolutional Layer 2
model.add(Conv2D(filters=36, kernel_size=5, strides=(2, 2), activation='relu'))

# Convolutional Layer 3
model.add(Conv2D(filters=48, kernel_size=5, strides=(2, 2), activation='relu'))

# Convolutional Layer 4
model.add(Conv2D(filters=64, kernel_size=3, strides=(1, 1), activation='relu'))

# Convolutional Layer 5
model.add(Conv2D(filters=64, kernel_size=3, strides=(1, 1), activation='relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('relu'))

model.add(Dense(1))

# Specify loss function and the optimizer for training
# we have used 'adam' optimizer with 'mean squared error' loss function
model.compile(loss='mse', optimizer='adam')


## 4. Train the model

In [112]:
import math

batch_size = 32

model.fit_generator(train_generator, steps_per_epoch=math.ceil(len(Sample_train)/batch_size), validation_data=validation_generator, validation_steps=math.ceil(len(Sample_valid)/batch_size), epochs=5, verbose=1)

# Save the model
model.save('model.h5')

print('Model Saved!')

# Print the model summary
model.summary()

Epoch 1/5
329/329 [==============================] - 119s 363ms/step - loss: 0.0215 - val_loss: 0.0167
Epoch 2/5
329/329 [==============================] - 63s 192ms/step - loss: 0.0142 - val_loss: 0.0137
Epoch 3/5
329/329 [==============================] - 63s 192ms/step - loss: 0.0120 - val_loss: 0.0124
Epoch 4/5
329/329 [==============================] - 63s 192ms/step - loss: 0.0104 - val_loss: 0.0120
Epoch 5/5
329/329 [==============================] - 63s 192ms/step - loss: 0.0096 - val_loss: 0.0125
Model Saved!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_24 (Lambda)           (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_23 (Cropping2D)   (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 31, 158, 24)       1824      
______________